<a href="https://colab.research.google.com/github/NVD1210/Sentiment-Analysis/blob/main/Question2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Question 2: CRAWL DATA**

# **Kết nối Colab với Drive**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
import os
os.chdir("/content")

Mounted at /content/drive


# **Import Libraries and Crawl Data**



Các bước thực hiện chính:

* Khởi tạo headers
* Gọi requests tới trang web cần thu thập
* Dùng BeautifulSoup để parse dữ liệu

In [ ]:
# Import Libraries
import requests
import re
from bs4 import BeautifulSoup

# Set up headers
headers = ({'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/44.0.2403.157 Safari/537.36',
            'Accept-Language': 'en-US, en;q=0.5'})
 
root_url = 'https://www.amazon.com'

apple_air_pod_url = '/Apple-MWP22AM-A-AirPods-Pro/dp/B07ZPC9QD4'

# requests
webpage = requests.get(root_url + apple_air_pod_url, headers=headers)

soup = BeautifulSoup(webpage.content, 'lxml', from_encoding='utf-8')

# **Lấy các thông tin về sản phẩm**

Các thông tin bao gồm:

* `model_name`: Tên sản phẩm
* `title`: Loại sản phẩm
* `brand`: Nhãn hiệu
* `color`: Màu sắc
* `connectivity_technology`: Khả năng kết nối
 

In [ ]:
title = soup.find("span", attrs={"id":"productTitle"}).string.strip()

ear_placement = soup.find("tr", attrs={'class':'a-spacing-small po-headphones_ear_placement'}).find("td", attrs={'class':'a-span9'}).find("span", attrs={'class':'a-size-base'}).string

color = soup.find("tr", attrs={'class':'a-spacing-small po-color'}).find("td", attrs={'class':'a-span9'}).find("span", attrs={'class':'a-size-base'}).string

brand = soup.find("tr", attrs={'class':'a-spacing-small po-brand'}).find("td", attrs={'class':'a-span9'}).find("span", attrs={'class':'a-size-base'}).string

connectivity_technology = soup.find("tr", attrs={'class':'a-spacing-small po-connectivity_technology'}).find("td", attrs={'class':'a-span9'}).find("span", attrs={'class':'a-size-base'}).string

model_name = soup.find("tr", attrs={'class':'a-spacing-small po-model_name'}).find("td", attrs={'class':'a-span9'}).find("span", attrs={'class':'a-size-base'}).string

In [ ]:
information_product = {'title': title, 'ear_placement': ear_placement, 'color':color, 
               'brand': brand, 'connectivity_technology': connectivity_technology, 'model_name':model_name}
               
information_product

{'brand': 'Apple',
 'color': 'White',
 'connectivity_technology': 'Wireless',
 'ear_placement': 'In Ear',
 'model_name': 'Apple AirPods Pro',
 'title': 'Apple AirPods Pro'}

# **Thu thập các đánh giá (reviews) của người dùng**

In [ ]:
# Các thông tin chính được thu thập:

# * `titles`: Nhận xét chung của người dùng
# * `reviews`: Nhận xét chi tiết của người dùng
# * `ratings`: Xếp hạng sản phẩm (từ 1 star - 5 star)


In [ ]:
all_reviews_url = soup.find('a', attrs={'data-hook':'see-all-reviews-link-foot'}).get('href')

In [ ]:
all_titles = []
all_reviews = []
all_ratings = []
all_years = []

In [ ]:
for k in range(501):

  if k == 402:
      continue
  
  all_reviews_r = requests.get(root_url + all_reviews_url + '&pageNumber=' + str(k), headers=headers)

  all_reviews_soup = BeautifulSoup(all_reviews_r.content, 'lxml', from_encoding='utf-8')

  try:
    # review - title
    for i in all_reviews_soup.findAll('a', attrs={'data-hook':'review-title'}):
      all_titles.append(i.find('span').string)
    
    # review - body
    for i in all_reviews_soup.findAll('span',attrs={'data-hook':'review-body'}):
      all_reviews.append(i.find('span').get_text(separator=" ").replace('\n','').strip())
    
    # review - star - rating
    for i in all_reviews_soup.findAll('i',attrs={'data-hook':'review-star-rating'}):
      all_ratings.append(re.findall(r'[\d.]+', i.find("span", attrs={'class':'a-icon-alt'}).string)[0])
    
    # review - date
    for i in all_reviews_soup.findAll('span',attrs={'data-hook':'review-date'}):
      all_years.append(re.findall(r'\d+', i.string)[1])
  
  except:
    print(f'Error when getting page {k}')

## **Kiểm tra việc Crawl**

In [ ]:
print("\nLength of titles: {} \
       \nLength of reviews: {} \
       \nLength of ratings: {}".format(all_titles, all_reviews, all_ratings))


# len(all_titles), len(all_reviews), len(all_ratings), len(all_years)


Length of titles: ['List of Features/Changes compared to Airpods (v2)', 'Saved my marriage', 'Apple forgot to pack a few components', 'Lit', 'Authentic and Beautiful, Love these way more than ever expected They are Great earphones', 'WOW', 'Hurt really bad. DO NOT BUY', 'Pro', 'Not worth it', 'Red Exclamation Already', 'List of Features/Changes compared to Airpods (v2)', 'Saved my marriage', 'Apple forgot to pack a few components', 'Lit', 'Authentic and Beautiful, Love these way more than ever expected They are Great earphones', 'WOW', 'Hurt really bad. DO NOT BUY', 'Pro', 'Not worth it', 'Red Exclamation Already', 'Excellent all around earbuds', 'Okay Apple', 'Transparency mode is game changer', 'Expensive garbage', 'Definitely worth the upgrade', 'AirPods vs AirPod Pro Review: My experiences', 'So uncomfortable, Sounds quality = Garbage... Over Hyped Trash', 'Not worth it .', 'Fake Airpods', 'I love this little AirPods pro', 'Disappointed', 'Do not buy from Amazon Warehouse any Appl

**Tạo 1 dataframe với dữ liệu đã thu thập**

In [ ]:
import pandas as pd

crawl_df = pd.DataFrame(list(zip(all_titles, all_reviews, all_ratings, all_years)),columns=['titles', 'reviews','ratings','years'])
crawl_df

,titles,reviews,ratings,years
0,List of Features/Changes compared to Airpods (v2),"Like iphone 11 Pro, Macbook Pro, Airpad Pro; t...",4.0,2019
1,Saved my marriage,"With the new transparency mode, I can listen t...",5.0,2019
2,Apple forgot to pack a few components,Love Apple products but I’m tired of their gam...,3.0,2019
3,Lit,They fire,5.0,2019
4,"Authentic and Beautiful, Love these way more t...",These earphones are top of the line. Here are ...,5.0,2019
...,...,...,...,...
4995,Surreal!,So freaking good! I never knew how cool this p...,5.0,2020
4996,Air Pro doesn’t come with a way to charge it!,Works fine. Now I just need to go out and buy ...,3.0,2021
4997,Worth Upgrade,"Sound quality is greatly improved on country, ...",5.0,2020
4998,These airpods pro are a total life changer.,The noise cancellation is beyond one of the wi...,5.0,2021


## **Xử lý Dữ liệu trùng lặp**

In [ ]:
# Check duplicate
crawl_df.duplicated().sum()

10

In [ ]:
# Delete duplicated row
crawl_df_update = crawl_df.drop_duplicates(ignore_index = True)
crawl_df_update.duplicated().sum()

0

## **Thông tin dữ liệu sau khi thu thập**

In [ ]:
crawl_df_update.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4990 entries, 0 to 4989
Data columns (total 4 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   titles   4990 non-null   object
 1   reviews  4990 non-null   object
 2   ratings  4990 non-null   object
 3   years    4990 non-null   object
dtypes: object(4)
memory usage: 156.1+ KB


# **Lưu Dữ liệu**

In [ ]:
crawl_df_update.to_csv('crawl.csv', index = False, header = True)

**Đọc dữ liệu từ file csv**

In [ ]:
df = pd.read_csv('crawl.csv')
df

,titles,reviews,ratings,years
0,List of Features/Changes compared to Airpods (v2),"Like iphone 11 Pro, Macbook Pro, Airpad Pro; t...",4.0,2019
1,Saved my marriage,"With the new transparency mode, I can listen t...",5.0,2019
2,Apple forgot to pack a few components,Love Apple products but I’m tired of their gam...,3.0,2019
3,Lit,They fire,5.0,2019
4,"Authentic and Beautiful, Love these way more t...",These earphones are top of the line. Here are ...,5.0,2019
...,...,...,...,...
4985,Surreal!,So freaking good! I never knew how cool this p...,5.0,2020
4986,Air Pro doesn’t come with a way to charge it!,Works fine. Now I just need to go out and buy ...,3.0,2021
4987,Worth Upgrade,"Sound quality is greatly improved on country, ...",5.0,2020
4988,These airpods pro are a total life changer.,The noise cancellation is beyond one of the wi...,5.0,2021
